## [詳解ディープラーニング　TensorFlow・Kerasによる時系列データ処理](https://book.mynavi.jp/ec/products/detail/id=72995)
　巣籠悠輔 著  
　マイナビ出版  
　ISBN : 978-4-8399-6251-7  
　発売 : 2017/05/30  
  
support site : [https://book.mynavi.jp/supportsite/detail/9784839962517.html](https://book.mynavi.jp/supportsite/detail/9784839962517.html)  
github : [yusugomori/deeplearning-tensorflow-keras](https://github.com/yusugomori/deeplearning-tensorflow-keras) 

---
### 第４章　続き

---
### Dropout
Overfittingなどの問題のため、深層学習では訓練データに対して予測精度を向上させたとしても  
評価データや実運用時の予測精度が上がるとは限らない。  
  
訓練データに含まれない未知のデータに対して予測精度を上昇させるように学習することを汎化（generalization）、  
また未知データに対しての予測性能を汎化性能（generalization ability）という。  
  
汎化のための手法はいくつも存在し、その一つがドロップアウト（dropout）である。    
これは学習時にランダムにニューロンをdropoutさせて出力を０にする手法で、  
擬似的なアンサンブル学習（個別に学習させた複数の識別器の出力から予測を作る）とみなすことができる。  
  
ドロップアウトの実装では対象の層と同じshapeを持つarrayであるマスクベクトル $m$ を用意し、  
学習時には出力との要素積をとり、出力時にはoff（or 全て１）とする。  

式で表すと、あるネットワークの入力 $x$ に対する出力 $h_{i}$ が  
  
\begin{align*}
h_{i} = f(Wx + b)
\end{align*}
  
と表されるとき、これを  

\begin{align*}
h_{i} = f(Wx + b) \odot m
\end{align*}

へと変更する。  
  
このとき続く層の出力 $ h_{i+1}$ を

\begin{align*}
h_{i+1} = g(Vh_{i} + c)
\end{align*}

また、それぞれの層で活性化関数に代入される重み付き和を

\begin{align*}
p := Wx + b \\
q := Vh_{i} + c
\end{align*}

とするとき、誤差項 $ \delta_{hi}, \delta_{hi+1} $ は、

\begin{align*}
\delta_{hi} &:= \frac{\partial E_{n}}{\partial p} \\
\delta_{hi+1} &:= \frac{\partial E_{n}}{\partial q}
\end{align*}

より

\begin{align*}
\delta_{hi} &= \frac{\partial E_{n}}{\partial q}\frac{\partial q}{\partial p} \\
&= \frac{\partial E_{n}}{\partial q}\frac{\partial}{\partial p}(Vf(p) \odot m + c) \\
&= f^{\prime}(p) \odot m \odot V^{\mathrm{T}}\delta_{hi+1}
\end{align*}

となる。  

---
#### Tensorflowでの実装例

In [5]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

class toy_test_tf_mnist(object):
    def __init__(self, data_size=10000, n_hidden_layer = 3, n_hidden_cell=200, 
                 train_size=0.8, test_size=0.2, learning_rate=0.01, np_seed=None):
        self.data_size = data_size
        self.n_hidden_layer = max(n_hidden_layer, 1)
        self.n_hidden_cell = max(n_hidden_cell, 1)
        self.train_size = train_size
        self.test_size = test_size
        self.lr = learning_rate
        self.np_seed = np_seed
        
    def set_each_dataset(self):
        mnist = fetch_mldata('MNIST original', data_home="../dataset")
        indices = np.random.permutation(len(mnist.data))[:self.data_size]
        self.X = mnist.data[indices]
        self.Y = mnist.target[indices]
        self.Y_onehot = np.eye(10)[self.Y.astype(int)]
        self.X_train, self.X_test, self.Y_train, self.Y_test = \
            train_test_split(self.X, self.Y_onehot, train_size=self.train_size, test_size=self.test_size)
        self.n_in, self.n_out = self.X_train[0].shape[0], self.Y_train[0].shape[0]

    def generate_model(self):
        self.x = tf.placeholder(tf.float32, shape=[None, self.n_in])
        self.t = tf.placeholder(tf.float32, shape=[None, self.n_out])
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.W = {}
        self.b = {}
        self.h = {}
        self.h_drop = {}
        
        # input layer - hidden layer 1
        self.W[0] = tf.Variable(tf.truncated_normal([self.n_in, self.n_hidden_cell], stddev=0.01))
        self.b[0] = tf.Variable(tf.zeros([self.n_hidden_cell]))
        self.h[0] = tf.nn.relu(tf.matmul(self.x, self.W[0]) + self.b[0])
        self.h_drop[0] = tf.nn.dropout(self.h[0], self.keep_prob)

        for i in range(self.n_hidden_layer - 1):
            self.W[i+1] = tf.Variable(tf.truncated_normal([self.n_hidden_cell, self.n_hidden_cell], stddev=0.01))
            self.b[i+1] = tf.Variable(tf.zeros([self.n_hidden_cell]))
            self.h[i+1] = tf.nn.relu(tf.matmul(self.h[i], self.W[i+1]) + self.b[i+1])
            self.h_drop[i+1] = tf.nn.dropout(self.h[i+1], self.keep_prob)

        # hidden layer - output layer
        self.W[self.n_hidden_layer] = tf.Variable(tf.truncated_normal([self.n_hidden_cell, self.n_out], stddev=0.01))
        self.b[self.n_hidden_layer] = tf.Variable(tf.zeros([self.n_out]))
        self.y = tf.nn.softmax(tf.matmul(self.h_drop[self.n_hidden_layer-1], self.W[self.n_hidden_layer]) + self.b[self.n_hidden_layer])

        self.cross_entropy = tf.reduce_mean(-tf.reduce_sum(self.t * tf.log(self.y), reduction_indices=[1]))
        self.train_step = tf.train.GradientDescentOptimizer(self.lr).minimize(self.cross_entropy)
        
        self.correct_prediction = tf.equal(tf.argmax(self.y, 1), tf.argmax(self.t, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction , tf.float32))
        
    def fit(self, epochs=30, batch_size=200):
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        
        n_batches = int(self.data_size * self.train_size) // batch_size
        
        for epoch in range(epochs):
            X_, Y_ = shuffle(self.X_train, self.Y_train)
            
            for i in range(n_batches):
                start = i * batch_size
                end = start + batch_size
            
                sess.run(self.train_step, feed_dict={
                    self.x: X_[start:end],
                    self.t: Y_[start:end],
                    self.keep_prob:0.5
                })
                
            loss = self.cross_entropy.eval(session=sess, feed_dict={
                self.x: X_,
                self.t: Y_,
                self.keep_prob: 1.0
            })
            acc = self.accuracy.eval(session=sess, feed_dict={
                self.x: X_,
                self.t: Y_,
                self.keep_prob: 1.0
            })
            print('epoch:', epoch, ' loss:', loss, ' accuracy:', acc)
                
        accuracy_rate = self.accuracy.eval(session=sess, feed_dict={
            self.x: self.X_test,
            self.t: self.Y_test,
            self.keep_prob: 1.0
        })
        print('accuracy: ', accuracy_rate)

    def predict(self):
        pass

In [6]:
testenv = toy_test_tf_mnist(data_size=10000, n_hidden_layer=3, n_hidden_cell=200, 
                 train_size=0.8, test_size=0.2, learning_rate=0.01, np_seed=None)
testenv.set_each_dataset()
testenv.generate_model()
testenv.fit(epochs=30, batch_size=200)

epoch: 0  loss: 2.25104  accuracy: 0.294
epoch: 1  loss: 1.60406  accuracy: 0.498375
epoch: 2  loss: 0.820454  accuracy: 0.739125
epoch: 3  loss: 0.655835  accuracy: 0.796
epoch: 4  loss: 0.496206  accuracy: 0.8485
epoch: 5  loss: 0.388966  accuracy: 0.8815
epoch: 6  loss: 0.352227  accuracy: 0.890375
epoch: 7  loss: 0.409333  accuracy: 0.864625
epoch: 8  loss: 0.281477  accuracy: 0.912375
epoch: 9  loss: 0.261775  accuracy: 0.920875
epoch: 10  loss: 0.223829  accuracy: 0.935375
epoch: 11  loss: 0.204165  accuracy: 0.937875
epoch: 12  loss: 0.189549  accuracy: 0.94575
epoch: 13  loss: 0.170097  accuracy: 0.951
epoch: 14  loss: 0.159841  accuracy: 0.950875
epoch: 15  loss: 0.143205  accuracy: 0.95775
epoch: 16  loss: 0.119476  accuracy: 0.967375
epoch: 17  loss: 0.127007  accuracy: 0.962625
epoch: 18  loss: 0.112417  accuracy: 0.9685
epoch: 19  loss: 0.0926286  accuracy: 0.974625
epoch: 20  loss: 0.0822767  accuracy: 0.977625
epoch: 21  loss: 0.0787187  accuracy: 0.978375
epoch: 22  los

In [7]:
# use double size of data
testenv_2 = toy_test_tf_mnist(data_size=20000, n_hidden_layer=3, n_hidden_cell=200, 
                 train_size=0.8, test_size=0.2, learning_rate=0.01, np_seed=None)
testenv_2.set_each_dataset()
testenv_2.generate_model()
testenv_2.fit(epochs=30, batch_size=200)

epoch: 0  loss: 1.41529  accuracy: 0.579625
epoch: 1  loss: 0.625784  accuracy: 0.791313
epoch: 2  loss: 0.379587  accuracy: 0.88925
epoch: 3  loss: 0.301959  accuracy: 0.910312
epoch: 4  loss: 0.252873  accuracy: 0.926375
epoch: 5  loss: 0.221352  accuracy: 0.936312
epoch: 6  loss: 0.179019  accuracy: 0.949188
epoch: 7  loss: 0.15513  accuracy: 0.955187
epoch: 8  loss: 0.145781  accuracy: 0.95875
epoch: 9  loss: 0.123266  accuracy: 0.965187
epoch: 10  loss: 0.121881  accuracy: 0.963812
epoch: 11  loss: 0.100241  accuracy: 0.970812
epoch: 12  loss: 0.0888309  accuracy: 0.975375
epoch: 13  loss: 0.0760668  accuracy: 0.9785
epoch: 14  loss: 0.0722346  accuracy: 0.979813
epoch: 15  loss: 0.0618083  accuracy: 0.983125
epoch: 16  loss: 0.0496171  accuracy: 0.987188
epoch: 17  loss: 0.0462618  accuracy: 0.988187
epoch: 18  loss: 0.0381531  accuracy: 0.991437
epoch: 19  loss: 0.0403054  accuracy: 0.990625
epoch: 20  loss: 0.0305167  accuracy: 0.9935
epoch: 21  loss: 0.0416628  accuracy: 0.987

---
#### Kerasでの実装例

In [10]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

from keras.utils.generic_utils import CustomObjectScope
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import tensorflow as tf
from keras.backend.tensorflow_backend import _to_tensor
from keras.backend.common import epsilon

def custom_categorical_crossentropy(target, output, from_logits=False, delta=1e-7):
    if not from_logits:
        output /= tf.reduce_sum(output,
                                axis=len(output.get_shape()) - 1,
                                keep_dims=True)
        _epsilon = _to_tensor(epsilon(), output.dtype.base_dtype)
        output = tf.clip_by_value(output, _epsilon, 1. - _epsilon)
        return - tf.reduce_sum(target * tf.log(output + delta),
                               axis=len(output.get_shape()) - 1)
    else:
        return tf.nn.softmax_cross_entropy_with_logits(labels=target,
                                                       logits=output)


class toy_test_keras_mnist(object):
    def __init__(self, data_size=10000, n_hidden_layer = 3, n_hidden_cell=200, 
                 train_size=0.8, test_size=0.2, learning_rate=0.01, keep_prob=0.5, np_seed=None):
        self.data_size = data_size
        self.n_hidden_layer = max(n_hidden_layer, 1)
        self.n_hidden_cell = max(n_hidden_cell, 1)
        self.train_size = train_size
        self.test_size = test_size
        self.lr = learning_rate
        self.keep_prob = keep_prob
        self.np_seed = np_seed
        
    def set_each_dataset(self):
        mnist = fetch_mldata('MNIST original', data_home="../dataset")
        indices = np.random.permutation(len(mnist.data))[:self.data_size]
        self.X = mnist.data[indices]
        self.Y = mnist.target[indices]
        self.Y_onehot = np.eye(10)[self.Y.astype(int)]
        self.X_train, self.X_test, self.Y_train, self.Y_test = \
            train_test_split(self.X, self.Y_onehot, train_size=self.train_size, test_size=self.test_size)
        self.n_in, self.n_out = self.X_train[0].shape[0], self.Y_train[0].shape[0]
        print("n_in={}, n_out={}".format(self.n_in, self.n_out))

    def fit(self, epochs=150, batch_size=200, activation_function='tanh', loss_function='categorical_crossentropy'):
        with CustomObjectScope({'custom_categorical_crossentropy':custom_categorical_crossentropy}):
            self.model = Sequential()

            self.model.add(Dense(self.n_hidden_cell, input_dim=self.n_in))
            self.model.add(Activation(activation_function))
            self.model.add(Dropout(self.keep_prob))

            for i in range(self.n_hidden_layer - 1):
                self.model.add(Dense(self.n_hidden_cell))
                self.model.add(Activation(activation_function))
                self.model.add(Dropout(self.keep_prob))

            self.model.add(Dense(self.n_out))
            self.model.add(Activation('softmax'))

            self.model.compile(loss = loss_function,
                               optimizer = SGD(lr=self.lr),
                               metrics=['accuracy']
                               )

            self.model.fit(self.X_train, self.Y_train, epochs=epochs, batch_size=batch_size)
            loss_and_metrics = self.model.evaluate(self.X_test, self.Y_test)
            print(loss_and_metrics)


In [12]:
testenv_keras = toy_test_keras_mnist(data_size=10000, n_hidden_layer = 3, n_hidden_cell=200, 
                                     train_size=0.8, test_size=0.2, 
                                     learning_rate=0.01, keep_prob=0.5, np_seed=None)
testenv_keras.set_each_dataset()
testenv_keras.fit(epochs=150, batch_size=200,
                  activation_function='tanh', loss_function='categorical_crossentropy')

n_in=784, n_out=10
Epoch 1/150
8000/8000 [==============================] - 0s 38us/step - loss: 2.7202 - acc: 0.1275
Epoch 2/150
8000/8000 [==============================] - 0s 26us/step - loss: 2.3500 - acc: 0.2032
Epoch 3/150
8000/8000 [==============================] - 0s 27us/step - loss: 2.1161 - acc: 0.2814
Epoch 4/150
8000/8000 [==============================] - 0s 27us/step - loss: 1.9114 - acc: 0.3368
Epoch 5/150
8000/8000 [==============================] - 0s 27us/step - loss: 1.7534 - acc: 0.3948
Epoch 6/150
8000/8000 [==============================] - 0s 29us/step - loss: 1.5834 - acc: 0.4584
Epoch 7/150
8000/8000 [==============================] - 0s 29us/step - loss: 1.4627 - acc: 0.4883
Epoch 8/150
8000/8000 [==============================] - 0s 30us/step - loss: 1.3699 - acc: 0.5194
Epoch 9/150
8000/8000 [==============================] - 0s 29us/step - loss: 1.2786 - acc: 0.5511
Epoch 10/150
8000/8000 [==============================] - 0s 26us/step - loss: 1.2251 - ac

8000/8000 [==============================] - 0s 27us/step - loss: 0.5258 - acc: 0.8395
Epoch 83/150
8000/8000 [==============================] - 0s 27us/step - loss: 0.5207 - acc: 0.8391
Epoch 84/150
8000/8000 [==============================] - 0s 28us/step - loss: 0.5233 - acc: 0.8389
Epoch 85/150
8000/8000 [==============================] - 0s 27us/step - loss: 0.5335 - acc: 0.8371
Epoch 86/150
8000/8000 [==============================] - 0s 30us/step - loss: 0.5139 - acc: 0.8444
Epoch 87/150
8000/8000 [==============================] - 0s 27us/step - loss: 0.5352 - acc: 0.8356
Epoch 88/150
8000/8000 [==============================] - 0s 25us/step - loss: 0.5140 - acc: 0.8385
Epoch 89/150
8000/8000 [==============================] - 0s 29us/step - loss: 0.5200 - acc: 0.8405
Epoch 90/150
8000/8000 [==============================] - 0s 30us/step - loss: 0.5082 - acc: 0.8435
Epoch 91/150
8000/8000 [==============================] - 0s 27us/step - loss: 0.5135 - acc: 0.8484
Epoch 92/150


実験時メモ：  
・Kerasで活性化関数をreluにすると高率に学習失敗する問題が発生。  
　自分のコードおよび書籍記載コードで発生を確認した。  
  
・本単元までで紹介された要素を対象として調査したが、この範囲ではバグの原因は不明で修正できず。  
　追試用コードではreluを使った学習はそれなりに成功するが、今回分のコードでは各10回試験して一度も学習できず。  
  
追記１：  
　Weightsの初期化部分で非ランダムな値を使うことで、問題が再現できることを確認した。  
　ランダムな初期値を明示的に与えると正しく学習できる。  
   
追記２：  
　docker上で gcr.io/tensorflow/tensorflow:latest-gpu のイメージを利用し学習したところ  
　正常に学習できる。（上のセルはこの環境における学習結果の一例）  
　versionが異なることもあるが、一部処理が異なるようだ。  
  　

---
#### Kerasにおける重みの初期化方法
　参考：[Keras Documentation | レイヤーの重み初期化方法](https://keras.io/ja/initializations/)  
  
・初期化用引数により、重みをランダムに初期化できる。  
  
~~~python
model.add(Dense(64, init='uniform'))
~~~

初期化方法一覧
- uniform
- lecun_uniform (LeCun 98)
- normal
- identity
- orthogonal
- zero
- glorot_normal (Glorot 2010)
- glorot_uniform
- he_normal (He et al., 2014)
- he_uniform

---
次のnotebookへ続く。